# [모듈 7.1] 배치 변환 스텝 개발

## 0. 기본 세이지 메이커 정보 및 기본 변수 로딩

In [21]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

%store -r 
%store

Stored variables and their in-db values:
base_preproc_input_dir                 -> 'opt/ml/processing/input'
dataset_path                           -> 'opt/ml/processing/input/dataset.csv'
default_bucket                         -> 'sagemaker-ap-northeast-2-057716757052'
image_uri                              -> '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com
input_data_uri                         -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
preprocessing_code_dir                 -> 'fraud/preprocessing.py'
processing_instance_count              -> ParameterInteger(name='ProcessingInstanceCount', p
project_prefix                         -> 'fraud2scratch'
s3_dataset_path                        -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
sagemaker_model                        -> 'pipelines-z2fiqp2pcp83-fraudscratchmodel-jvft2mid
test_preproc__dir_artifact             -> 's3://sagemaker-ap-northeast-2-057716757052/sklear
train_model_artifact                   -> 's3:

## 로컬 모드

In [ ]:
# from sagemaker.transformer import Transformer
# import pandas as pd

# instance_type = 'local'
# output_prefix = 'batch-output'
# output_path = 's3://{}/{}/{}'.format(default_bucket, project_prefix, output_prefix)

# transformer = Transformer(base_transform_job_name = "FraudScratchBatchTransform",
#                           model_name = sagemaker_model,
#                           instance_count=1,
#                           instance_type= instance_type,
#                           output_path= output_path
#                          )

# transformer.transform(test_preproc__dir_artifact,
#                       content_type='text/csv'
#                      )
# # transformer.wait()

## 1. 로컬에서 스크립트 실행

### 환경 셋업

In [26]:
import os

def split_X_y(test_preproc_path):
    df = pd.read_csv(test_preproc_path)
    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)
    
    return y_test, df

s3_batch_test_path = f"{s3_dataset_path}/batch"
print("s3_batch_test_path: ", s3_batch_test_path)


# 전처리된 테스트 데이터 생성
test_preproc_path = f"{test_preproc__dir_artifact}/test.csv"
y_test, test_batch_df = split_X_y(test_preproc_path)

# 로컬에 배치 변환 테스트 데이터 폴더 및 경로 생성 
base_preproc_input_batch_dir = 'opt/ml/processing/input/batch'
os.makedirs(base_preproc_input_batch_dir, exist_ok=True)
batch_test_path = f"{base_preproc_input_batch_dir}/batch.csv"
print("batch_test_path: ", batch_test_path)

# 로컬에 배치 변환 파일 저장
test_batch_df.to_csv(batch_test_path, index=None)

# S3에 업로드
input_batch_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=batch_test_path, 
    desired_s3_uri=s3_batch_test_path,
)
print("input_batch_data_uri: ", input_batch_data_uri)

s3_batch_test_path:  s3://sagemaker-ap-northeast-2-057716757052/fraud2scratch/input/batch
batch_test_path:  opt/ml/processing/input/batch/batch.csv
input_batch_data_uri:  s3://sagemaker-ap-northeast-2-057716757052/fraud2scratch/input/batch/batch.csv


In [28]:
print("sagemaker_model: \n ", sagemaker_model)

sagemaker_model: 
  pipelines-z2fiqp2pcp83-fraudscratchmodel-jvft2midf1


## 3. 모델 빌딩 파이프라인에서  실행 
---



### 모델 빌딩 파이프라인 변수 생성



In [11]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

### 배치추론 실행을 위해 배치 변환단계 정의 

이제 모델 인스턴스가 정의되었습니다. 다음으로 `Transformer`인스턴스를 정의합니다. 적절한 모델 타입과 실행할 인스턴스타입, 출력이 저장될 S3 URI를 지정합니다. 


구체적으로는 `CreateModelStep`단계의 `step_create_model` 인스턴스의 속성 중 `ModelName`을 이용하여 모델명을 전달하였습니다. `CreateModelStep`의 `properties`속성은 [DescribeModel](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeModel.html) API의 응답 오브젝트와 대응됩니다. 


In [12]:
batch_data = ParameterString(
    name="BatchData",
    default_value=input_batch_data_uri,
)

batch_transform_output = f"s3://{default_bucket}/{project_prefix}"
print("batch_transform_output: \n", batch_transform_output)

batch_transform_output: 
 s3://sagemaker-ap-northeast-2-057716757052/fraud2scratch


In [29]:
from sagemaker.transformer import Transformer


transformer = Transformer(
    model_name= sagemaker_model,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    output_path= batch_transform_output
)

In [30]:
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep


step_transform = TransformStep(
    name="FraudScratchTransform",
    transformer=transformer,
    inputs=TransformInput(data= batch_data,  
                           split_type='Line',
                          content_type = "text/csv")
)

### 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의



In [31]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
#         processing_instance_type, 
#         processing_instance_count,
        batch_data
    ],
    steps=[step_transform],
)

#### (선택) 파이프라인 정의 확인 

파이프라인을 정의하는 JSON을 생성하고 파이프라인 내에서 사용하는 파라미터와 단계별 속성들이 잘 정의되었는지 확인할 수 있습니다.

In [32]:
import json


definition = json.loads(pipeline.definition())
# definition

### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [33]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/fraud2scratch',
 'ResponseMetadata': {'RequestId': 'ef9ac205-1b76-48c5-8d89-58f1f39703ac',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ef9ac205-1b76-48c5-8d89-58f1f39703ac',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Tue, 13 Apr 2021 12:15:37 GMT'},
  'RetryAttempts': 0}}

In [34]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [35]:
execution.describe()
# execution.wait()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/fraud2scratch',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/fraud2scratch/execution/tjn7tdn4m8v7',
 'PipelineExecutionDisplayName': 'execution-1618316138411',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 4, 13, 12, 15, 38, 288000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 4, 13, 12, 15, 38, 288000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '9d07393a-3ab0-4e7c-bcca-319326a8dd46',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9d07393a-3ab0-4e7c-bcca-319326a8dd46',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '401',
   'date': 'Tue, 13 Apr 2021 12:15:38 GMT'},
  'RetryAttempts': 0}}

In [36]:
execution.list_steps()

[{'StepName': 'FraudScratchTransform',
  'StartTime': datetime.datetime(2021, 4, 13, 12, 15, 39, 55000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'Metadata': {'TransformJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:transform-job/pipelines-tjn7tdn4m8v7-fraudscratchtransfor-gwrsalii3q'}}}]